In [1]:
%reset -f
import sys
sys.path.append("/home/jose/Documents/Maestria/Codigo/OTI/build/")
import Calc_Cable as cab
import pyoti.sparse as oti
import numpy as np
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
import pyoti.core as coti

In [38]:
#Lectura de datos de entrada
l,h,L0,A0,E,w,cargas = cab.leer_entradas("Irvine_1")

#Pertubar parametros (no sirve para perturbar cargas y ubicacion s de las cargas)
order = 1
nbases = 6
e1 = oti.e(1, order=order)
e2 = oti.e(2, order=order)
e3 = oti.e(3, order=order)
e4 = oti.e(4, order=order)
e5 = oti.e(5, order=order)
e6 = oti.e(6, order=order)
#l = l + e1
#h = h + e2
L0 = L0 + e3 
#A0 = A0 + e4
#E = E + e5
#w = w + e6

#convertir adimensionales
gamma,delta,beta,Psi,Sigmas = cab.param_adim(l,h,L0,A0,E,w,cargas)

#calcular las incognitas gi fi
gi_0, fi_0 = 0.3, 0.4 #valores iniciales para iterar en newton
gi,fi = cab.calc_gi_fi(order,nbases,gamma,delta,beta,Psi,Sigmas,gi_0,fi_0)

#Reacciones

X = gi*w*L0
Y0 = fi*w*L0

#dX/dL0 , dY0/dL0
d = 3
print("gi, fi")
print(gi.get_deriv(0), fi.get_deriv(0))
print(gi.get_deriv(d), fi.get_deriv(d))

#dX/dL0 , dY0/dL0
d = 3
print("X, Y0")
print(X.get_deriv(0), Y0.get_deriv(0))
print(X.get_im(d), Y0.get_im(d))

#puntos de la carga 1
n = 1
xi, eta, tau = cab.solu(gi,fi,beta,Psi,Sigmas,Sigmas[n],n)
x = xi*L0
y = eta*L0

#dx/dL0 , dy/dL0
# d = 3
# print("x, y")
# print(x.get_deriv(0), y.get_deriv(0))
# print(x.get_deriv(d), y.get_deriv(d))

gi, fi
[[0.48072921]] [[0.6621794]]
[[-1.75770757]] [[-0.00149706]]
X, Y0
[[24.03646052]] [[33.10896976]]
[[-67.85499479]] [[27.51595534]]


In [14]:
def root(x0, ht=0):
    l,h,L0,A0,E,w,cargas = cab.leer_entradas("Irvine_1")
    L0 += 1j*ht
    gamma,delta,beta,Psi,Sigmas = param_adim(l,h,L0,A0,E,w,cargas)
    root = fsolve(real_f, x0, args=(gamma,delta,beta,Psi,Sigmas))
    XY0 = np.array([root[0] + 1j*root[1], root[2] + 1j*root[3]])*w*L0
    return XY0

In [31]:
def param_adim(l,h,L0,A0,E,w,cargas):
    W = w*L0
    gamma = l/L0
    delta = h/L0
    beta = W/(E*A0)
    Psi = np.append([0,0],cargas[:,0])/W
    Sigmas = np.append([0],cargas[:,1])/L0
    return (gamma,delta,beta,Psi,Sigmas)

In [32]:
def func(x,gamma,delta,beta,Psi,Sigmas):
    f = np.zeros(2, dtype="complex")
    sum_gi = 0
    sum_fi = 0
    for i in range(len(Sigmas)):
        sum_gi = sum_gi + np.arcsinh((x[1]-Psi[i+1]-Sigmas[i])/x[0]) - np.arcsinh((x[1]-Psi[i]-Sigmas[i])/x[0])
        sum_fi = sum_fi + beta*Psi[i+1]*(Sigmas[i]-1) + np.sqrt((x[1]-Psi[i+1]-Sigmas[i])**2 + x[0]**2) - np.sqrt((x[1]-Psi[i]-Sigmas[i])**2 + x[0]**2)
    f[0] = beta - gamma/x[0] + np.arcsinh(x[1]/x[0]) - np.arcsinh((x[1]-np.sum(Psi)-1)/x[0]) + sum_gi
    f[1] = beta*(x[1]-0.5) - delta + np.sqrt(x[1]**2+x[0]**2) - np.sqrt((x[1]-np.sum(Psi)-1)**2 + x[0]**2) + sum_fi
    return f

In [33]:
def real_f(x0, gamma,delta,beta,Psi,Sigmas):
    r1, c1, r2, c2 = x0
    x = np.array([r1 + 1j*c1, r2 + 1j*c2])
    f_4 = func(x,gamma,delta,beta,Psi,Sigmas)
    return np.array([np.real(f_4[0]), np.imag(f_4[0]), np.real(f_4[1]), np.imag(f_4[1])])

In [40]:
x0 = np.array([0.3, 0, 0.4, 0])
ht = 1e-8

r = root(x0,ht)
print("(X, Y0) = ", np.real(r))
print("(dX/dL0, dY0/dL0) = ", np.imag(r)/ht)

(X, Y0) =  [24.03646052 33.10896976]
(dX/dL0, dY0/dL0) =  [-67.24632462  27.50894951]
